In [1]:
using Random
using Distributions
using FLoops
using Base.Threads
using BenchmarkTools

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [2]:
dtype = Float32
seed = 123

# DATA CONFIG

alpha = 0.05

# data
B  = 100   # num. coverage probabilities per boxplot
S  = 4300  # num. samples per coverage probability
nx = 15   # size of group 1
ny = 5    # size of group 2
MC = 10_000
perms = nothing
if binomial(nx + ny, nx) < 30_000
    perms = partition(nx, ny)
end

([1 1 … 5 6; 2 2 … 7 7; … ; 14 14 … 19 19; 15 16 … 20 20], [20 20 … 6 5; 19 19 … 4 4; … ; 17 17 … 2 2; 16 15 … 1 1])

In [66]:
maximum(mean.(distrX))

13.906618f0

In [68]:
# POPULATION SETTINGS

Random.seed!(123)

distrTypeX = Gamma{dtype}
X_shape = random(Uniform(2, 4), B)
X_scale = random(Uniform(0.5, 2), B)
distrX = map(distrTypeX, X_shape, X_scale)

distrTypeY = Gumbel{dtype}
Y_loc = random(Uniform(0, 1), B)
Y_scale = random(Uniform(2, 4), B)
distrY = map(distrTypeY, Y_loc, Y_scale)
;

In [69]:
deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = Gamma{Float32}[Gamma{Float32}(α=3.813f0, θ=1.931f0), Gamma{Float32}(α=2.887f0, θ=1.769f0)]
distrY[1:2] = Gumbel{Float32}[Gumbel{Float32}(μ=0.965f0, θ=2.056f0), Gumbel{Float32}(μ=0.071f0, θ=2.495f0)]
deltas[1:2] = Float32[5.211148, 3.5959504]


In [70]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


5×4300 Matrix{Float32}:
  2.68864   1.96555  -0.581105  -4.38383   …   5.63533   -2.71138  -1.13305
 -1.33976  -1.46723  -1.54114    4.69909       4.1404     1.8591    1.50926
  2.24906   1.10736   4.06871    3.98353       1.85863   12.6145    4.14539
  2.27932   8.47562   3.28491    0.795135     -0.930118   9.36958   5.35393
  2.29428  -1.40773   2.66228   -3.13632       5.59606    2.75996   6.20328

In [71]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

5×4300 Matrix{Float32}:
  2.68864   1.96555  -0.581105  -4.38383   …   5.63533   -2.71138  -1.13305
 -1.33976  -1.46723  -1.54114    4.69909       4.1404     1.8591    1.50926
  2.24906   1.10736   4.06871    3.98353       1.85863   12.6145    4.14539
  2.27932   8.47562   3.28491    0.795135     -0.930118   9.36958   5.35393
  2.29428  -1.40773   2.66228   -3.13632       5.59606    2.75996   6.20328

In [72]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []

    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end
    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [73]:
T = Threads.nthreads()
results = Array{Union{Tuple, Nothing}, 3}(nothing, 6, B, S)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
 
    results[1, b, s] = permInterval(x, y, deltas[b], perms, true, alpha, twoSided, twoSided, 0.0005)    
    results[2, b, s] = permInterval(x, y, deltas[b], perms, false, alpha, twoSided, twoSided, 0.0005)
    results[3, b, s] = permInterval(x, y, deltas[b], perms, true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], perms, false, alpha/2, greater, smaller, 0.0005)
    
    """
    results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    """
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/3/"
save_permutation_results(results, B, S; dir=dir)
#save_ci_results(results, 6, B, S; prefix="bs", dir=dir)

4848.103651 seconds (3.49 G allocations: 52.512 TiB, 84.04% gc time, 0.00% compilation time)
dir * filename = "../results/15_5/3/twoSided_pooled_0.05.csv"
dir * filename = "../results/15_5/3/twoSided_unpooled_0.05.csv"
dir * filename = "../results/15_5/3/oneSided_pooled_0.025.csv"
dir * filename = "../results/15_5/3/oneSided_unpooled_0.025.csv"
